# Msc Thesis

 ## > Building an FR model

## LIBRARY IMPORTS

In [13]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import cv2
import sys
import tensorflow as tf
from tensorflow.keras import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, LocallyConnected2D, Flatten, Dense, Dropout

## DEFINE THE MODEL ARCHITECTURE

In [2]:
def build_model():
	model = Sequential()
	model.add(Conv2D(32, (11, 11), activation='relu', name='C1', input_shape=(152, 152, 3)))
	model.add(MaxPooling2D(pool_size=3, strides=2, padding='same', name='M2'))
	model.add(Conv2D(16, (9, 9), activation='relu', name='C3'))
	model.add(LocallyConnected2D(16, (9, 9), activation='relu', name='L4'))
	model.add(LocallyConnected2D(16, (7, 7), strides=2, activation='relu', name='L5') )
	model.add(LocallyConnected2D(16, (5, 5), activation='relu', name='L6'))
	model.add(Flatten(name='F0'))
	model.add(Dense(4096, activation='relu', name='F7'))
	model.add(Dropout(rate=0.5, name='D0'))
	model.add(Dense(8631, activation='softmax', name='F8'))
	
	return model

## LOAD WEIGHTS FROM DEEPFACE VGGFACE2 PRETRAINED MODEL

In [4]:
#Can be downloaded from https://github.com/swghosh/DeepFace/releases
model = build_model()
model.load_weights(r"C:\Users\Keravnos\Documents\AdversarialFR\VGGFace2_DeepFace_weights_val-0.9034.h5")

## REMOVE REPRESENTATION LAYER

In [9]:
deepface_model = Model(inputs=model.layers[0].input, outputs=model.layers[-3].output)

## COMPARE REPRESENTATIONS

In [11]:
def l2_normalize(x):
    return x / np.sqrt(np.sum(np.multiply(x, x)))
 
def findEuclideanDistance(source_representation, test_representation):
    euclidean_distance = source_representation - test_representation
    euclidean_distance = np.sum(np.multiply(euclidean_distance, euclidean_distance))
    euclidean_distance = np.sqrt(euclidean_distance)
    return euclidean_distance

In [12]:
img1_embedding = deepface_model.predict("img1.jpg")[0]
img2_embedding = deepface_model.predict("img2.jpg")[0]
 
euclidean_l2_distance = findEuclideanDistance(l2_normalize(img1_embedding)
, l2_normalize(img2_embedding))

IndexError: list index out of range

## EXTRACTING FACIAL FEATURES

In [17]:
def extract_features_from_img(imagePath):
    image = cv2.imread(imagePath)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    faceCascade = cv2.CascadeClassifier(cv2.data.haarcascades +         "haarcascade_frontalface_default.xml")
    faces = faceCascade.detectMultiScale(
        gray,
        scaleFactor=1.3,
        minNeighbors=3,
        minSize=(30, 30)
                )
    print("Found {0} Faces!".format(len(faces)))
    for (x, y, w, h) in faces:
        cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
        roi_color = image[y:y + h, x:x + w]
        print("[INFO] Object found. Saving locally.")
        cv2.imwrite(str(w) + str(h) + '_faces.jpg', roi_color)
    # MIGHT HAVE TO CHANGE THIS SO IT ONLY LOADS IT IN MEMORY INSTEAD OF SAVING IT.
    status = cv2.imwrite('faces_detected.jpg', image)
    print ("Image faces_detected.jpg written to filesystem: ",status)